I am sharing my script for calculating my CV score. In this notebook, I use hungarian matching algorithm to maximize F1 score. I hope there won't be much different between mine and competition metric. Please feel free to correct me if I make a mistake.

By using this script I get a CV of 0.368 and LB 0.4

In [ ]:
import numpy as np
from scipy.optimize import linear_sum_assignment
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

In [ ]:
def iou(bbox1, bbox2):
    bbox1 = [float(x) for x in bbox1]
    bbox2 = [float(x) for x in bbox2]

    (x0_1, y0_1, x1_1, y1_1) = bbox1
    (x0_2, y0_2, x1_2, y1_2) = bbox2

    # get the overlap rectangle
    overlap_x0 = max(x0_1, x0_2)
    overlap_y0 = max(y0_1, y0_2)
    overlap_x1 = min(x1_1, x1_2)
    overlap_y1 = min(y1_1, y1_2)

    # check if there is an overlap
    if overlap_x1 - overlap_x0 <= 0 or overlap_y1 - overlap_y0 <= 0:
            return 0

    # if yes, calculate the ratio of the overlap to each ROI size and the unified size
    size_1 = (x1_1 - x0_1) * (y1_1 - y0_1)
    size_2 = (x1_2 - x0_2) * (y1_2 - y0_2)
    size_intersection = (overlap_x1 - overlap_x0) * (overlap_y1 - overlap_y0)
    size_union = size_1 + size_2 - size_intersection

    return size_intersection / size_union

In [ ]:
def precision_calc(gt_boxes, pred_boxes):
    cost_matix = np.ones((len(gt_boxes), len(pred_boxes)))
    for i, box1 in enumerate(gt_boxes):
        for j, box2 in enumerate(pred_boxes):
            dist = abs(box1[0]-box2[0])
            if dist > 4:
                continue
            iou_score = iou(box1[1:], box2[1:])

            if iou_score < 0.35:
                continue
            else:
                cost_matix[i,j]=0

    row_ind, col_ind = linear_sum_assignment(cost_matix)
    fn = len(gt_boxes) - row_ind.shape[0]
    fp = len(pred_boxes) - col_ind.shape[0]
    tp=0
    for i, j in zip(row_ind, col_ind):
        if cost_matix[i,j]==0:
            tp+=1
        else:
            fp+=1
            fn+=1
    return tp, fp, fn

# Testing algorithm
boxes format: frame_index, xmin, ymin, xmax, ymax

In [ ]:
gt_boxes = [[0, 50,60, 120, 130], [0, 40, 20, 110, 80], [0, 140, 20, 190, 80]]
pred_boxes = [[0, 55, 30, 130, 110], [0, 60, 90, 135, 140], [0, 70, 120, 155, 190]]

im = np.array(Image.new('RGB', (224, 224)))
fig,ax = plt.subplots(1)
ax.imshow(im)

for box in gt_boxes:
    rect = patches.Rectangle((box[1],box[2]),box[3]-box[1],box[4]-box[2],linewidth=1,edgecolor='g',facecolor='none')
    ax.add_patch(rect)
    
for box in pred_boxes:
    rect = patches.Rectangle((box[1],box[2]),box[3]-box[1],box[4]-box[2],linewidth=1,edgecolor='r',facecolor='none')
    ax.add_patch(rect)

In [ ]:
tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
print(f'TP: {tp}, FP: {fp} FN: {fn}')

# Calculating CV score

In [ ]:
#Calculating CV score.
testdata = np.load('../input/test-metrics/testdata.npy', allow_pickle=True)
ftp, ffp, ffn = [], [], []
for count, data in enumerate(testdata):
    pred_boxes = data['data']['preds']
    gt_boxes = data['data']['gt']
    tp, fp, fn = precision_calc(gt_boxes, pred_boxes)
    ftp.append(tp)
    ffp.append(fp)
    ffn.append(fn)

tp = np.sum(ftp)
fp = np.sum(ffp)
fn = np.sum(ffn)
precision = tp / (tp + fp + 1e-6)
recall =  tp / (tp + fn +1e-6)
f1_score = 2*(precision*recall)/(precision+recall+1e-6)
print(f'TP: {tp}, FP: {fp}, FN: {fn}, PRECISION: {precision:.4f}, RECALL: {recall:.4f}, F1 SCORE: {f1_score}')